In [ ]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score
import os
from pandas import read_csv

In [ ]:
df = pd.read_csv("../input/bostonhoustingmlnd/housing.csv")

In [ ]:
df.head(10)

* RM : nombre de pièces en moyenne
* LSTAT : % bas de la population définit par le niveau d'étude
* PTRATIO : ratio jeune/professeur par ville
* MEDV : Valeur médianne des maisons occupées en Dollars US

In [ ]:
df.info()

In [ ]:
df.count()

RECHERCHE DE CORRELATION

In [ ]:
tabcorr = df.corr()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

Regroupement des paramètres par clusters classés par proximité

In [ ]:
sns.clustermap(abs(tabcorr), cmap="coolwarm")

Dendrogramme des corrélations entre les caractéristiques :

In [ ]:
from scipy.cluster import hierarchy as hc

corr = 1 - df.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=df.columns, orientation='left', leaf_font_size=10)


On s'intéresse surtout à la corrélation par rapport à la valeur des maisons :

In [ ]:
correlations = tabcorr.MEDV
print(correlations)

In [ ]:
correlations = correlations.drop(['MEDV'],axis=0)

In [ ]:
print(abs(correlations).sort_values(ascending=False))

On voit que la plus forte corrélation est LSTAT, suivie de RM et enfin PTRATIO

**Régression linéaire multiple**

In [ ]:
X = df.drop(['MEDV'], axis=1)
y = df.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

On trace le nuage de points pour comparer la prédiction et les résultats attendus

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

Visualisation de la distribution de l'erreur avec seaborn

In [ ]:
sns.distplot(y_test-y_pred)

Calcule l'erreur sur les moindres carrés

In [ ]:
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

**Régression par forêts aléatoires**

In [ ]:
X = df.drop(['MEDV'], axis=1)
y = df.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_rf)))

**XGB Regressor**

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")